### Course: Bayesian Filtering and Smoothing.
### Exercised round: 6
### Student's name: Diego Alejandro Agudelo España.

### Exercise 1. (Optimal Importance Distribution)


Recall the following state space model from Exercise 3 on Round 1:

\begin{align}
	\mathbf{x}_k &= \begin{bmatrix} 1 & 1 \\ 0 & 1 \end{bmatrix} \mathbf{x}_{k-1} + \mathbf{w}_{k-1} \\
	y_k   &= \begin{bmatrix} 1 & 0 \end{bmatrix} \mathbf{x}_{k-1} + v_{k}	
\end{align}


where $\mathbf{x}_k = \begin{bmatrix}x_k & \dot{x}_k \end{bmatrix}^\top$ is the state, $y_k$ is the measurement, and $\mathbf{w}_k \sim \mathcal{N}(\mathbf{0}, \operatorname{diag}(1/10^2,1^2))$ and $v_k \sim \mathcal{N}(0,10^2)$ are white Gaussian noise processes.


1. Write down the Kalman filter equations for this model.
2. Derive expression for the optimal importance distribution for the model:

	\begin{align}
		\pi(\mathbf{x}_k) = p(\mathbf{x}_k|\mathbf{x}_{k-1},\mathbf{y}_{1:k}).
	\end{align}
    
3. Write pseudo code for the corresponding particle filter algorithm (sequential importance resampling algorithm). Also write down the equations for the weight update.
4. Compare the number of CPU steps (multiplications and additions) needed by the particle filter and Kalman filter. Which implementation would you choose for a real implementation?

### Solution Exercise 1.

### 1.

Rewriting the given state space model as:

\begin{align}
	\mathcal{x}_k &= \mathbf{A} \mathcal{x}_{k-1} + \mathcal{w}_{k-1} \qquad & \mathcal{w}_{k-1} \sim \mathcal{N}(0,\mathbf{Q}) \\
	y_k   &= \mathbf{H} \mathcal{x}_{k-1} + v_{k} \qquad & v_{k}  \sim \mathcal{N}(0, R)
\end{align}

where

$$
\mathbf{A} =  \begin{bmatrix} 1 & 1 \\ 0 & 1 \end{bmatrix} \qquad \mathbf{H} =  \begin{bmatrix} 1 & 0 \end{bmatrix} \qquad \mathbf{Q} = \operatorname{diag}(1/10^2,1^2) \qquad R = 10^2
$$



The Kalman Filter equations for the aforementioned model are the following:





* Initialization:

$$
\mathbf{x}_0  \sim \mathcal{N}(\mathbf{m}_0, \mathbf{P}_0)
$$

* Prediction Step:

$$
\begin{align}
    \mathbf{m}_k^{-} & = \mathbf{A} \mathbf{m}_{k-1}\\
    \mathbf{P}_k^{-} & = \mathbf{A} \mathbf{P}_{k-1} \mathbf{A}^\top + \mathbf{Q}
\end{align}
$$


* Update Step

$$
\begin{align}
    \mathbf{S}_k & = \mathbf{H} \mathbf{P}_{k}^{-} \mathbf{H}^\top + \mathbf{R} \\
    \mathbf{K}_k & = \mathbf{P}_{k}^{-} \mathbf{H}^\top \mathbf{S}_k^{-1}\\
    \mathbf{m}_k & = \mathbf{m}_k^{-} + \mathbf{K}_k (\mathbf{y}_k - \mathbf{H} \mathbf{m}_k^{-} ) \\
    \mathbf{P}_k & = \mathbf{P}_k^{-} - \mathbf{K}_{k} \mathbf{S}_k \mathbf{K}_{k}^\top
\end{align}
$$

### 2.

Deriving the optimal importance distribution for the given state space model

$$
\begin{align}
		\pi(\mathbf{x}_k) = p(\mathbf{x}_k|\mathbf{x}_{k-1},\mathbf{y}_{1:k}).
\end{align}
$$

Given the markovian property over the dynamics we have

$$
p(\mathbf{x}_k|\mathbf{x}_{k-1},\mathbf{y}_{1:k}) = p(\mathbf{x}_k|\mathbf{x}_{k-1},{y}_{k})
$$

Now using Bayes theorem we can rewrite this expression as:

$$
\begin{align}
p(\mathbf{x}_k|\mathbf{x}_{k-1},y_{k}) & = \frac{p(y_k | \mathbf{x}_k, \mathbf{x}_{k-1}) p(\mathbf{x}_k | \mathbf{x}_{k-1})}{ p(y_k | \mathbf{x}_{k-1})  } \\
                                                   & = \frac{p(y_k | \mathbf{x}_k) p(\mathbf{x}_k | \mathbf{x}_{k-1})}{ p(y_k | \mathbf{x}_{k-1})  }
\end{align}
$$

Recall that the probabilities $p(y_k | \mathbf{x}_k)$ and $ p(\mathbf{x}_k | \mathbf{x}_{k-1})$ are Gaussian distributed, namely:


$$
p(y_k | \mathbf{x}_k) \sim \mathcal{N}(\mathbf{H} \mathbf{x}_{k}, R) \qquad p(\mathbf{x}_k | \mathbf{x}_{k-1})  \sim \mathcal{N}(\mathbf{A} \mathbf{x}_{k-1}, \mathbf{Q})
$$

Therefore Gaussian properties can be applied and the joint distribution of random variables $\mathbf{x}_k$ and $y_k$ given $\mathbf{x}_{k-1}$ can be expressed as:

$$
\begin{pmatrix}
\mathbf{x}_k \\
\mathbf{y}_k \\
\end{pmatrix}
\sim
\mathcal{N}
\bigg(
\begin{pmatrix}
\mathbf{A} \mathbf{x}_{k-1} \\
\mathbf{H} \mathbf{A} \mathbf{x}_{k-1}  \\
\end{pmatrix}
,
\begin{pmatrix}
\mathbf{Q} &   \mathbf{Q} \mathbf{H}^\top \\
  \mathbf{H} \mathbf{Q}  & \mathbf{R} + \mathbf{H} \mathbf{Q} \mathbf{H}^\top \\
\end{pmatrix}
\bigg)
$$

Now conditioning on $y_k$ we get:

$$
p(\mathbf{x}_k|\mathbf{x}_{k-1},{y}_{k}) = \mathcal{N}(\mathbf{m}_{\pi}, \mathbf{P}_{\pi})
$$

where

$$
\mathbf{m}_{\pi} = \mathbf{A} \mathbf{x}_{k-1} + \mathbf{Q} \mathbf{H}^\top  (\mathbf{R} + \mathbf{H} \mathbf{Q} \mathbf{H}^\top)^{-1} (y_k - \mathbf{H} \mathbf{A} \mathbf{x}_{k-1})
$$

$$
\mathbf{P}_{\pi} = \mathbf{Q} - \mathbf{Q} \mathbf{H}^\top  (\mathbf{R} + \mathbf{H} \mathbf{Q} \mathbf{H}^\top)^{-1} \mathbf{H} \mathbf{Q}
$$

### 3.

In order to implement the sequential importance resampling algorithm (SIR) the weights update need to be derived. Recall that general weight update for SIR takes the following form:

$$
W_k^i \propto W_{k-1}^i \frac{p(y_k^i | \mathbf{x}_k^i) p(\mathbf{x}_k^i | \mathbf{x}_{k-1}^i)}{\pi(\mathbf{x}_k^i | \mathbf{x}_{0:k-1}^i,  \mathbf{y}_{1:k}^i)}
$$

Taking into account the particular form of the optimal important distribution we assumed in this exercise, an equivalent weight update will be:

$$
\begin{align}
W_k^i & \propto W_{k-1}^i \frac{p(y_k^i | \mathbf{x}_k^i) p(\mathbf{x}_k^i | \mathbf{x}_{k-1}^i)}{ \bigg( \frac{p(y_k^i | \mathbf{x}_k^i) p(\mathbf{x}_k^i | \mathbf{x}_{k-1}^i)}{ p(y_k | \mathbf{x}_{k-1}^i)  } \bigg)} \\
\\
      & \propto W_{k-1}^i  p(y_k | \mathbf{x}_{k-1}^i) 
\end{align}
$$



Using the joint distribution derived for $\mathbf{x}_k$ and $y_k$ given $\mathbf{x}_{k-1}$ we can easily get the marginal distribution $p(y_k | \mathbf{x}_{k-1}^i)$ which is:

$$
p(y_k | \mathbf{x}_{k-1}^i) = \mathcal{N}( y_k | \mathbf{H} \mathbf{A} \mathbf{x}_{k-1}^i,  \mathbf{H} \mathbf{Q} \mathbf{H}^\top + \mathbf{R})
$$

Now we have all we need to describe a SIR algorithm for this particular model.

* Let $T$ be the length of the measurements sequence.
* Let $N$ be the number of particles used.
* for $k$ in $1,2, \dots, T$:
    * for $i$ in $1,2, \dots,N$:
        * Draw $\mathbf{x}_k^i$ from important distribution $p(\mathbf{x}_k^i|\mathbf{x}_{k-1}^i,{y}_{k}) = \mathcal{N}(\mathbf{m}_{\pi}, \mathbf{P}_{\pi})$
        * Calculate the new weight using $W_{k}^i \propto W_{k-1}^i  \mathcal{N}( y_k | \mathbf{H} \mathbf{A} \mathbf{x}_{k-1}^i,  \mathbf{H} \mathbf{Q} \mathbf{H}^\top + \mathbf{R})$ 
    * Normalize the weights $W_{k}^i$ to sum to unity.
    * if the number of effective particles is too low:
        * do resampling


### 4.

Since most of the computational operations are given in terms of matrix multiplications we have to estimate the number of CPU steps this operation takes. Let's assume that the number of steps (i.e additions and multiplications) needed to multiply a matrix $\mathbf{A}$ of size $a \times b$ by a matrix $\mathbf{B}$ of size $b \times c$ is about $2 a b c$. Let's also assume that the cost of summing of two matrices $\mathbf{A}$ and $\mathbf{B}$ of length $a \times b$ is $ab$.

Let's start by giving an estimation of the number of CPU steps needed to peform a Kalman filter iteration.

* Prediction step:

$$
\begin{align}
    \mathbf{m}_k^{-} & = \mathbf{A} \mathbf{m}_{k-1} \qquad \text{steps} = 2(2\times2\times1) = 8\\
    \mathbf{P}_k^{-} & = \mathbf{A} \mathbf{P}_{k-1} \mathbf{A}^\top + \mathbf{Q} \qquad \text{steps} = 2(2\times2\times2) + 2(2\times2\times2) + 2 \times 2 = 36
\end{align}
$$

* Update step.

$$
\begin{align}
    \mathbf{S}_k & = \mathbf{H} \mathbf{P}_{k}^{-} \mathbf{H}^\top + \mathbf{R} \qquad \text{steps} = 2(1\times2\times2) + 2(1\times2\times1) + 1 = 13\\
    \mathbf{K}_k & = \mathbf{P}_{k}^{-} \mathbf{H}^\top \mathbf{S}_k^{-1} \qquad \text{steps} = 2(2\times2\times1) + 2 = 10\\
    \mathbf{m}_k & = \mathbf{m}_k^{-} + \mathbf{K}_k (\mathbf{y}_k - \mathbf{H} \mathbf{m}_k^{-} ) \qquad \text{steps} = 2(1\times2\times1) + 5 = 9 \\
    \mathbf{P}_k & = \mathbf{P}_k^{-} - \mathbf{K}_{k} \mathbf{S}_k \mathbf{K}_{k}^\top  \qquad \text{steps} = 2(2\times1\times2) + 6 = 14
\end{align}
$$

The above estimation takes into account that some values are in fact scalar values. This gives an estimation of roughly $90$ CPU steps per Kalman filter iteration. Assuming that we have a measurements sequence of length $T$ the overall estimation will be of $90T$

On the other hand the particle filter is largely dependent on the number of particles $N$ used to represent the posterior distribution. Most of the matrix multiplication operations can be done offline since they don't depend on $\mathbf{x}_k$ or $\mathbf{y}_k$. A rough estimation of the CPU operations needed by the particle filter is $20 N T + C$ where $20$ is the estimation of steps performed for each particle in each iteration and $C$ represents the offline computational work.

Relying on this estimation it becomes clear that the computational load per iteration of the particle filter is greater to its counterpart using a few particles (e.g. more than $5$). Therefore the Kalman Filter is a better choice from the perspective of computational efficiency.